In [1]:
include("nn_ops.jl")

maxpoolconstraint

In [2]:
batch = 1
in_height = 5
in_width = 5
stride_height = 2
stride_width = 2
pooled_height = round(Int, in_height/stride_height, RoundUp)
pooled_width = round(Int, in_width/stride_width, RoundUp)
in_channels = 1
filter_height = 2
filter_width = 2
out_channels = 4

srand(2)
x_actual = rand(-10:10, batch, in_height, in_width, in_channels)
x_current = rand(-10:10, batch, in_height, in_width, in_channels)
filter = rand(-10:10, filter_height, filter_width, in_channels, out_channels)
x_conv_actual = conv2d(x_actual, filter)
x_conv_relu_actual = relu(x_conv_actual)
x_conv_relu_maxpool_actual = maxpool(x_conv_relu_actual, (1, 2, 2, 1));

In [3]:
m = Model(solver=GurobiSolver())

@variable(m, ve[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m, vx[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m, vx_conv[1:batch, 1:in_height, 1:in_width, 1:out_channels])
@variable(m, vx_conv_relu[1:batch, 1:in_height, 1:in_width, 1:out_channels])
@variable(m, vx_conv_relu_maxpool[1:batch, 1:pooled_height, 1:pooled_width, 1:out_channels])
@constraint(m, vx .== x_current) # input

# 1. Only using convolution constraint
@constraint(m, conv2d(vx+ve, filter) .== vx_conv)
# @constraint(m, vx_conv .== x_conv_actual)

# 2. Adding relu layer
reluconstraint2(m, vx_conv, vx_conv_relu, 10000)
# @constraint(m, vx_conv_relu .== x_conv_relu_actual)

# 3. Adding maxpool layer
maxpoolconstraint(m, vx_conv_relu, vx_conv_relu_maxpool, (2, 2), 10000)
@constraint(m, vx_conv_relu_maxpool .== x_conv_relu_maxpool_actual)


@objective(m, Min, sum(ve.^2))

:Min

In [4]:
status = solve(m)

println("Objective value: ", getobjectivevalue(m))
# TODO: Are jump solutions global? Can I save particular variables?
println("e = ", getvalue(ve))

# TODO: fix the error message WARNING: A variable named a is already attached to this model. If creating variables programmatically, consider using the anonymous variable syntax x = @variable(m, [1:N], ...).

Optimize a model with 797 rows, 486 columns and 2209 nonzeros
Model has 25 quadratic objective terms
Variable types: 286 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 443 rows and 203 columns
Presolve time: 0.00s
Presolved: 354 rows, 283 columns, 1010 nonzeros
Presolved model has 25 quadratic objective terms
Variable types: 151 continuous, 132 integer (132 binary)
Found heuristic solution: objective 1458.0643602

Root relaxation: objective 8.547419e+01, 481 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   85.47419    0   56 1458.06436   85.47419  94.1%     -    0s
H    0     0                     222.2162142   85.47419  61.5%     -    0s
     0   

In [ ]:
x_conv_current = conv2d(x_current+getvalue(ve), filter)
x_conv_relu_current = relu(x_conv_current)
x_conv_relu_maxpool_current = maxpool(x_conv_relu_current, (1, 2, 2, 1));

In [ ]:
x_conv_current - x_conv_actual

In [ ]:
x_conv_relu_current - x_conv_relu_actual

In [ ]:
x_conv_relu_maxpool_current - x_conv_relu_maxpool_actual

In [ ]:
typeof(vx)

In [ ]:
getoutputsize(x_actual, (1, 2, 2, 1))

In [ ]:
collect(CartesianRange((1, 3, 3, 1)))

In [ ]:
(1, 2, 3, 4) == (1, 2, 3, 4)

In [ ]:
maximum(x_conv_relu_maxpool_current - x_conv_relu_maxpool_actual)